In [41]:
import sys
import os 
import openmc
import numpy as np
import matplotlib.pyplot as plt
import openmc.mgxs as mgxs
import pandas as pd
from IPython.display import Image
import glob

### Import Output datas

In [42]:
def load_mgxs(sp_file_path,sm_file_path,model_file_path):
    sm=openmc.Summary(sm_file_path)
    sp = openmc.StatePoint(sp_file_path)
    sp.link_with_summary(sm)
    model=openmc.model.Model
    model=openmc.model.Model.from_model_xml(model_file_path)
    mgxs_lib = openmc.mgxs.Library(model.geometry)
    groups = openmc.mgxs.EnergyGroups(openmc.mgxs.GROUP_STRUCTURES['XMAS-172'])
    mgxs_lib.energy_groups = groups
    mgxs_lib.scatter_format = "legendre"
    mgxs_lib.legendre_order = 3
    mgxs_lib.mgxs_types = ['total', 'absorption', 'fission', 'nu-fission', 'chi', 'scatter matrix', 'nu-scatter matrix', 'inverse-velocity']
    mgxs_lib.tally_trigger =openmc.Trigger('std_dev', 1e-2)
    mgxs_lib.by_nuclide = False
    mgxs_lib.domain_type = 'cell'
    mgxs_lib.domains = model.geometry.get_all_material_cells().values()
    mgxs_lib.build_library()
    mgxs_lib.load_from_statepoint(sp)
    return(mgxs_lib)

In [44]:
sp_6_families_path='/Users/pablogarcia44/repo/perso/openmc/17x17_assembly/17x17_guide_tube/sp_6_families.h5'
sm_6_families_path='/Users/pablogarcia44/repo/perso/openmc/17x17_assembly/17x17_guide_tube/sm_6_families.h5'
model_6_families_path='/Users/pablogarcia44/repo/perso/openmc/17x17_assembly/17x17_guide_tube/model_6_families.xml'
mgxs_lib_6_families=load_mgxs(sp_6_families_path,sm_6_families_path,model_6_families_path)

sp_1_family_path='/Users/pablogarcia44/repo/perso/openmc/17x17_assembly/17x17_guide_tube/sp_1_family.h5'
sm_1_family_path='/Users/pablogarcia44/repo/perso/openmc/17x17_assembly/17x17_guide_tube/sm_1_family.h5'
model_1_family_path='/Users/pablogarcia44/repo/perso/openmc/17x17_assembly/17x17_guide_tube/model_1_family.xml'
mgxs_lib_1_family=load_mgxs(sp_1_family_path,sm_1_family_path,model_1_family_path)

/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=7.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=8.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=9.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Surface instance already exists with id=109.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Surface instance already exists with id=110.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Surface instance already exists with id=111.
  warn(msg, IDWarning)
/

### Post Processing

In [45]:
def cell_domain(cell,mgxs_lib):
    for k in range(len(mgxs_lib.domains)):
        if mgxs_lib.domains[k].name==cell:
            return(mgxs_lib.domains[k].id)

In [46]:
def mgxs_value(cell,reaction,mgxs_lib):  
    cell_name=cell
    cell=cell_domain(cell_name,mgxs_lib)
    mgxs_plot = mgxs_lib.get_mgxs(cell,reaction)
    mgxs=mgxs_plot.get_pandas_dataframe(xs_type='macro')['mean'].values
    return(mgxs)

In [66]:
def rd_max(mgxs1,mgxs2):   
    avg=0
    index=0
    rd_max=0
    for i in range (171):
        rd1=0
        rd2=0
        if mgxs1[i]!=0:
            rd1=abs(mgxs1[i]-mgxs2[i])/mgxs1[i]
        if mgxs2[i]!=0:
            rd2=abs(mgxs1[i]-mgxs2[i])/mgxs2[i]
        if rd1>rd_max or rd1>rd_max:
            rd_max=max(rd1,rd2)
            index=i
    return(100*rd_max,index)      

In [69]:
fuel_1=mgxs_value('fuel_1_family','fission',mgxs_lib_1_family)
fuel_E=mgxs_value('fuel_E','fission',mgxs_lib_6_families)
fuel_C=mgxs_value('fuel_C','fission',mgxs_lib_6_families)
fuel_N=mgxs_value('fuel_N','fission',mgxs_lib_6_families)


print(rd_max(fuel_N,fuel_C))
print(rd_max(fuel_N,fuel_E))
print(rd_max(fuel_E,fuel_C))

(9.081669831081102, 79)
(3.55279605429432, 64)
(7.794569216902131, 79)
